# Chapter 45: Reasoning Models - ReAct and Chain-of-Thought
Structured reasoning loops for AI agents.


In [ ]:
import json
print("Reasoning Models ready")


In [ ]:
steps=["Check if IP is in correct subnet.","Verify default gateway via ping.","Check routing table for prefix.","Inspect ACLs for ICMP blocks."]
print("Question: Why is this host unreachable?")
for i,s in enumerate(steps,1): print(f"Step {i}: {s}")
print("Answer: ACL on edge router blocking ICMP.")


In [ ]:
class ReActAgent:
    def __init__(self,n=5): self.n=n; self.trace=[]
    def think(self,obs): return f"Analyzing: {obs[:40]}"
    def act(self,t):
        if "ping" in t.lower(): return "ping_host",{"host":"10.0.0.1"}
        if "routing" in t.lower(): return "check_route",{"pfx":"0.0.0.0/0"}
        return "log",{"note":t[:40]}
    def observe(self,a,p):
        if a=="ping_host": return "Ping: 3 lost, 1 received. High loss."
        if a=="check_route": return "No route in FIB. Routing issue confirmed."
        return "Logged note"
    def run(self,prob):
        obs=prob
        for step in range(self.n):
            t=self.think(obs); a,p=self.act(t); obs=self.observe(a,p)
            self.trace.append({"step":step+1,"action":a,"obs":obs[:60]})
            if "confirmed" in obs: break
        return self.trace
agent=ReActAgent()
for t in agent.run("Host 10.0.0.5 unreachable"): print(f"Step {t[chr(115)+chr(116)+chr(101)+chr(112)]}: [{t[chr(97)+chr(99)+chr(116)+chr(105)+chr(111)+chr(110)]}] {t[chr(111)+chr(98)+chr(115)]}")


In [ ]:
def plan(problem,tools):
    steps=[{"id":i,"tool":t,"status":"pending"} for i,t in enumerate(["get_logs","ping_host","get_config"],1)]
    for s in steps: s["status"]="done" if s["tool"] in tools else "skipped"
    done=sum(1 for s in steps if s["status"]=="done")
    print(json.dumps({"problem":problem,"steps":steps,"conclusion":f"{done}/{len(steps)} completed"},indent=2))
plan("BGP flapping on PE-Router-1",["get_logs","ping_host"])
